## Sentiment Analysis.py 
----
The purpose of this code is to analyze the sentiment of any tweeter user input into this file to then correlate it bitcoin price. The goal is to be able to find users who can relatively predict the price of bitcoin up to a few days later.

Libraries will include, numpy, pandas, matplotlib, tweepy, seaborn, and plotly.



## Getting Bitcoin price history

In [690]:
import pandas as pd
import numpy as np
#import pandas_profiling

import seaborn as sns
import matplotlib.pyplot as plt
get_ipython().magic('matplotlib inline')
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True) 

import time
import matplotlib.dates as mdates
#import datetime as dt
from datetime import datetime


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# get market info for bitcoin from the start of 2013 to the current day
bitcoin_df = pd.read_html("https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20130428&end="+time.strftime("%Y%m%d"))[0]

# convert the date string to the correct date format
bitcoin_df = bitcoin_df.assign(Date=pd.to_datetime(bitcoin_df['Date']))

# when Volume is equal to '-' convert it to 0
#does not work: bitcoin_df.loc[bitcoin_df['Volume'] == '-', 'Volume']=0
bitcoin_df['Volume'].replace('-','0',inplace=True)

# convert to int
bitcoin_df['Volume'] = bitcoin_df['Volume'].astype('int64')

# sort values from most recent to January 2017
bitcoin_df = bitcoin_df.sort_values('Date',ascending=True)

#change to same datetime as twitter df
bitcoin_df['Date'] = bitcoin_df['Date'].dt.strftime('%Y/%m/%d')

bitcoin_df = bitcoin_df.rename(columns={'Market Cap': 'Market_Cap'})


In [691]:
bitcoin_2017_df = bitcoin_df[(bitcoin_df['Date']>'2017-01-01')]
bitcoin_2017_df['Change'] = bitcoin_2017_df['Close'].diff()
bitcoin_2017_df = bitcoin_2017_df.iloc[1:]

# For Up or Down in Price
change_dummy = []
for i, row in bitcoin_2017_df.iterrows():
    if row.Change >= 0:
        pos = 1
        change_dummy.append(pos)
        
    else:
        neg = 0
        change_dummy.append(neg)

bitcoin_2017_df['Change_dummy'] = change_dummy
bitcoin_2017_df.tail()

/Users/MichaelRubio/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,Date,Open,High,Low,Close,Volume,Market_Cap,Change,Change_dummy
4,2018/04/16,8337.57,8371.15,7925.73,8058.67,5631310000,141571000000,-270.44,0
3,2018/04/17,8071.66,8285.96,7881.72,7902.09,6900880000,137070000000,-156.58,0
2,2018/04/18,7944.43,8197.80,7886.01,8163.42,6529910000,134926000000,261.33,1
1,2018/04/19,8159.27,8298.69,8138.78,8294.31,7063210000,138591000000,130.89,1
0,2018/04/20,8286.88,8880.23,8244.54,8845.83,8438110000,140777000000,551.52,1


## Importing CSV data from previous sentiments analysis

In [692]:
sunnarborg_df = pd.read_csv("analysis/Alex Sunnarborg-137-tweets.csv")
sunnarborg_df = sunnarborg_df[["tweet_source", "tweet_date", "tweet_vader_score", "tweet_text"]]
sunnarborg_df = sunnarborg_df.groupby(['tweet_source','tweet_date']).mean()
sunnarborg_df.reset_index(drop=False, inplace=True)
sunnarborg_df.sort_values('tweet_date', inplace=True)
sunnarborg_df.reset_index(drop=True, inplace=True)
sunnarborg_df = sunnarborg_df.rename(columns={'tweet_vader_score': 'mean_tweet_vader_score'})
sunnarborg_df['two_day_mov_avg'] = sunnarborg_df['mean_tweet_vader_score'].rolling(window=3).mean()
sunnarborg_df['seven_day_mov_avg'] = sunnarborg_df['mean_tweet_vader_score'].rolling(window=7).mean()
sunnarborg_df = sunnarborg_df.rename(columns={'tweet_date': 'Date'})
sunnarborg_df = sunnarborg_df.iloc[7:]
sunnarborg_df = sunnarborg_df.rename(columns={'tweet_date': 'Date'})
sunnarborg_df = sunnarborg_df.rename(columns={'mean_tweet_vader_score': 'sunnarborg_mean_score'})

In [693]:
adresen_df = pd.read_csv("analysis/Gavin Andresen-241-tweets.csv")
adresen_df = adresen_df[["tweet_source", "tweet_date", "tweet_vader_score", "tweet_text"]]
adresen_df = adresen_df.groupby(['tweet_source','tweet_date']).mean()
adresen_df.reset_index(drop=False, inplace=True)
adresen_df.sort_values('tweet_date', inplace=True)
adresen_df.reset_index(drop=True, inplace=True)
adresen_df = adresen_df.rename(columns={'tweet_vader_score': 'mean_tweet_vader_score'})
adresen_df['two_day_mov_avg'] = adresen_df['mean_tweet_vader_score'].rolling(window=3).mean()
adresen_df['seven_day_mov_avg'] = adresen_df['mean_tweet_vader_score'].rolling(window=7).mean()
adresen_df = adresen_df.rename(columns={'tweet_date': 'Date'})
adresen_df = adresen_df.iloc[7:]
adresen_df = adresen_df.rename(columns={'mean_tweet_vader_score': 'adresen_mean_score'})

In [694]:
novogratz_df = pd.read_csv("analysis/Michael Novogratz-142-tweets.csv")
novogratz_df = novogratz_df[["tweet_source", "tweet_date", "tweet_vader_score", "tweet_text"]]
novogratz_df = novogratz_df.groupby(['tweet_source','tweet_date']).mean()
novogratz_df.reset_index(drop=False, inplace=True)
novogratz_df.sort_values('tweet_date', inplace=True)
novogratz_df.reset_index(drop=True, inplace=True)
novogratz_df = novogratz_df.rename(columns={'tweet_vader_score': 'mean_tweet_vader_score'})
novogratz_df = novogratz_df.rename(columns={'tweet_date': 'Date'})
novogratz_df['two_day_mov_avg'] = novogratz_df['mean_tweet_vader_score'].rolling(window=3).mean()
novogratz_df['seven_day_mov_avg'] = novogratz_df['mean_tweet_vader_score'].rolling(window=7).mean()
novogratz_df = novogratz_df.rename(columns={'tweet_date': 'Date'})
novogratz_df = novogratz_df.iloc[7:]
novogratz_df = novogratz_df.rename(columns={'mean_tweet_vader_score': 'novogratz_mean_score'})

In [695]:
final_df = pd.merge(bitcoin_2017_df, sunnarborg_df,  on='Date', how='outer')
final_df = pd.merge(final_df, adresen_df, on='Date', how='outer')
final_df = pd.merge(final_df, novogratz_df, on='Date', how='outer')
final_df.fillna(0, inplace=True)
final_df = final_df[(final_df['Date'] > '2017-01-01')]
final_df = final_df[["Date", "Close", "Change", "Change_dummy","sunnarborg_mean_score", "adresen_mean_score", "novogratz_mean_score", "Volume"]] #"Volume", "Market_Cap"
final_df.tail(5)

,Date,Close,Change,Change_dummy,sunnarborg_mean_score,adresen_mean_score,novogratz_mean_score,Volume
469,2018/04/16,8058.67,-270.44,0.0,0.0,0.0,0.00000,5.631310e+09
470,2018/04/17,7902.09,-156.58,0.0,0.0,0.0,0.31845,6.900880e+09
471,2018/04/18,8163.42,261.33,1.0,0.0,0.0,0.42725,6.529910e+09
472,2018/04/19,8294.31,130.89,1.0,0.0,0.0,0.00000,7.063210e+09
473,2018/04/20,8845.83,551.52,1.0,0.0,0.0,0.00000,8.438110e+09


In [696]:
# average score between users
final_df['avg_score'] = (final_df['sunnarborg_mean_score'] + final_df['adresen_mean_score'] + final_df['novogratz_mean_score']) / 3

final_df = final_df[final_df.avg_score != 0]

final_df.head()

,Date,Close,Change,Change_dummy,sunnarborg_mean_score,adresen_mean_score,novogratz_mean_score,Volume,avg_score
28,2017/01/30,920.38,0.88,1.0,0.4528,0.11505,0.0,78227300.0,0.189283
29,2017/01/31,970.40,50.02,1.0,0.0000,0.03680,0.0,164582000.0,0.012267
30,2017/02/01,989.02,18.62,1.0,0.0000,0.18762,0.0,150110000.0,0.062540
31,2017/02/02,1011.80,22.78,1.0,0.5188,0.00000,0.0,145821000.0,0.172933
33,2017/02/04,1042.90,12.99,1.0,0.0000,0.47530,0.0,155064000.0,0.158433


In [697]:
days_skipped = input("How many days ahead shall we predict? ")
days_skipped = int(days_skipped)

final_df.Date = final_df.Date.shift(days_skipped) ## shift down
final_df.Close = final_df.Close.shift(days_skipped) ## shift down
final_df.Change = final_df.Change.shift(days_skipped) ## shift down
final_df.Change_dummy = final_df.Change_dummy.shift(days_skipped) ## shift down
final_df.Volume = final_df.Volume/1000000
#final_df.Market_Cap = final_df.Market_Cap/1000000000000
final_df.head(5)

How many days ahead shall we predict? 1


,Date,Close,Change,Change_dummy,sunnarborg_mean_score,adresen_mean_score,novogratz_mean_score,Volume,avg_score
28,NaN,NaN,NaN,NaN,0.4528,0.11505,0.0,78.2273,0.189283
29,2017/01/30,920.38,0.88,1.0,0.0000,0.03680,0.0,164.5820,0.012267
30,2017/01/31,970.40,50.02,1.0,0.0000,0.18762,0.0,150.1100,0.062540
31,2017/02/01,989.02,18.62,1.0,0.5188,0.00000,0.0,145.8210,0.172933
33,2017/02/02,1011.80,22.78,1.0,0.0000,0.47530,0.0,155.0640,0.158433


In [698]:
final_df.sort_values('Date', inplace=True)
final_df = final_df.iloc[180:-days_skipped]
final_df.head(days_skipped + 1)

,Date,Close,Change,Change_dummy,sunnarborg_mean_score,adresen_mean_score,novogratz_mean_score,Volume,avg_score
324,2017/11/21,8071.26,-129.38,0.0,0.0,0.0,0.42792,3633.53,0.14264
325,2017/11/22,8253.55,182.29,1.0,0.0,0.0,0.66915,4225.18,0.22305


In [699]:
# to export to csv

#final_df.to_csv("combined_df2.csv", encoding="utf-8")

## Getting Bitcoin price history

In [700]:
target = final_df["Change_dummy"]
target_names = ["Up", "Down"]

In [701]:
final_df.drop(['Date'], axis = 1, inplace = True)
final_df.drop(['Close'], axis = 1, inplace = True)
final_df.drop(['Change'], axis = 1, inplace = True)
final_df.drop(['Change_dummy'], axis = 1, inplace = True)
final_df.drop(['avg_score'], axis = 1, inplace = True)

data = final_df
data.head(1)

,sunnarborg_mean_score,adresen_mean_score,novogratz_mean_score,Volume
324,0.0,0.0,0.42792,3633.53


In [702]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [703]:
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [704]:
# One-day shift forward

In [705]:
# Model Accuracy
print('Train Acc: %.3f' % model.score(X_train, y_train))

Train Acc: 0.551


In [706]:
# Model Accuracy
print('Test Acc: %.3f' % model.score(X_test, y_test))

Test Acc: 0.667
